In [7]:
import pandas as pd    
import numpy as np 
import csv
import datetime

In [17]:
rawData = pd.read_csv(r'../data/rawData/' + '2020CobbCountyEvictionCaseEvents-SCRAPE-09-18-2020' + '.csv')

#create dict  {caseid:[{date: , name: }]}
answerDict = {}

for index, row in rawData.iterrows():
    
    if row['Case.ID'] not in answerDict:
        answerDict[row['Case.ID']] = []
        
    if row['Event.Name'].lower().find('answer') != -1:
        answerDict[row['Case.ID']].append({'date': row['Event.Date'],'name':row['Event.Name']})

validAnswerDict = {key:answerDict[key] for key in answerDict if len(answerDict[key]) > 0}
# sort each valid answer case by date
for key in validAnswerDict:
    validAnswerDict[key].sort(key=lambda x: datetime.datetime.strptime(x['date'],'%m/%d/%y'),reverse = True)
# print(validAnswerDict)


rawDataIDAddressOnly = rawData.drop(['Event.Number','Event.Date','Event.Name'], axis=1)
rawDataIDAddressOnly.head()
rawDataIDAddressOnlyUnique = rawDataIDAddressOnly.drop_duplicates().reset_index(drop=True)

rawDataIDAddressOnlyUnique['answerDate'] = 'N/A'
rawDataIDAddressOnlyUnique['ifAnswered'] = 0
rawDataIDAddressOnlyUnique['answerEventName'] = 'N/A'

for index, row in rawDataIDAddressOnlyUnique.iterrows():
    
    if row['Case.ID'] in validAnswerDict:
        rawDataIDAddressOnlyUnique.at[index,'answerDate'] = validAnswerDict[row['Case.ID']][0]['date']
        rawDataIDAddressOnlyUnique.at[index,'ifAnswered'] = 1
        rawDataIDAddressOnlyUnique.at[index,'answerEventName'] = validAnswerDict[row['Case.ID']][0]['name']

# rawDataIDAddressOnlyUnique.head(20)
rawDataIDAddressOnlyUnique.to_csv('../data/test/'+ '2020CobbCountyEvictionCaseEvents-SCRAPE-09-18-2020' + '.csv')